### Network Design Example

This example implements a warehouse model: In this project, I had the opportunity to work on a network design optimization task. The primary goal was to efficiently assign warehouses to Origins in order to minimize fixed costs, variable costs, inventory costs, and transportation costs. The project aimed at achieving a well-balanced and cost-effective distribution network

In [1]:
# installing the gurobipy and gurobipy pandas
%pip install gurobipy
%pip install gurobipy_pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 27.0 MB/s eta 0:00:00


Importing all the necessary libraries

In [2]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import sys
import gurobipy_pandas as gppd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#### Uploading Dataset - Origin.csv, Demand_data - Sheet1.csv, Orig_Dest.csv & Input_Factors.xlsx

initially read in Origin data. The origin data contain 4 columns: origin, fixed_cost, var_cost and cap and set the origin column as an index, origin(location) where warehouse's could be installesd

In [3]:
FilePath = "/content/drive/MyDrive/TCS_assighnment/Network_Design/Network_d.xlsx"
print(FilePath)

/content/drive/MyDrive/TCS_assighnment/Network_Design/Network_d.xlsx


In [4]:
xls=pd.ExcelFile(FilePath)
print(xls.sheet_names)

['Origin', 'Demand_data', 'Orig_dst', 'Input_Factors']


In [5]:
# Read the Excel file into a Pandas DataFrame
# # load the Origin worksheet
warehouse_data = pd.read_excel(xls, 'Origin', index_col='origin')
warehouse_data

,fixed_cost,var_cost,cap
origin,,,
chennai,50000,2.23,400000
bhopal,56000,2.32,240000
jaipur,65000,2.25,300000
kolkata,65000,2.45,200000
nasik,55000,2.21,300000
pune,89000,2.56,300000
sonipat,95000,2.78,240000


In [6]:
# check tha data type
warehouse_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, chennai to sonipat
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   fixed_cost  7 non-null      int64  
 1   var_cost    7 non-null      float64
 2   cap         7 non-null      int64  
dtypes: float64(1), int64(2)
memory usage: 224.0+ bytes


In [7]:
# removing the ',' and '$' from this table
warehouse_data = warehouse_data.replace({',': '', '\$': ''}, regex=True)
warehouse_data

,fixed_cost,var_cost,cap
origin,,,
chennai,50000,2.23,400000
bhopal,56000,2.32,240000
jaipur,65000,2.25,300000
kolkata,65000,2.45,200000
nasik,55000,2.21,300000
pune,89000,2.56,300000
sonipat,95000,2.78,240000


converting the "fixed_cost" & "cap" column in the "warehouse_data" DataFrame to integer data type. This is done using the astype(int) method, which is a Pandas method for changing the data type of a column.

In [8]:
warehouse_data["fixed_cost"] = warehouse_data["fixed_cost"].astype(int)
warehouse_data["cap"] = warehouse_data["cap"].astype(int)

# converting the "var_cost" column in the "warehouse_data" DataFrame to float data type
warehouse_data["var_cost"] = warehouse_data["var_cost"].astype(float)

Moreover read the Demand_data: in this data set we are havinh 2 columns ask_dem and destination where we have to transport from origin. destination column set as an index

In [9]:
# load the Demand_data worksheet
demand_data = pd.read_excel(xls, 'Demand_data', index_col='destination')
demand_data.head()

,ask_dem
destination,
chennai,28000
bhopal,35000
bhubaneshwar,29500
jaipur,37500
kochi,34000


In [10]:
demand_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13 entries, chennai to srinagar
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   ask_dem  13 non-null     int64
dtypes: int64(1)
memory usage: 208.0+ bytes


In addition read the Orig_dst data. in this data set we are having 3 columns origin, destination, and kms, kms(distance) origin to destination

In [11]:
# load the Orig_dst worksheet
Orig_Dest = pd.read_excel(xls, 'Orig_dst')
Orig_Dest.head()

,origin,destination,kms
0,chennai,chennai,100.0
1,chennai,bhopal,1170.1
2,chennai,bhubaneshwar,994.8
3,chennai,jaipur,1605.6
4,chennai,kochi,556.1


In [12]:
Orig_Dest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   origin       90 non-null     object 
 1   destination  90 non-null     object 
 2   kms          90 non-null     float64
dtypes: float64(1), object(2)
memory usage: 2.2+ KB


At the read the Input_Factors data from this data set we extract "big M Factor", "Transportation Cost per unit Per Km" and "Max # Warehouses allowed"

In [13]:
# load the Input_Factors worksheet
input_factors = pd.read_excel(xls, 'Input_Factors')
input_factors

,Input Factors,Data,Units
0,big M Factor,500000.00,NaN
1,Max # Warehouses allowed,7.00,NaN
2,Avg Review Period,12.00,Weeks
3,Avg Lead Time,8.00,Weeks
4,Avg Safety Stock,4.00,Weeks
5,Avg Unit Price,150.00,$
6,Avg Inventory Holding Cost,0.15,percent
7,Transportation Cost per unit Per Km,0.02,$


In [14]:
max_warehouses = int(input_factors.loc[input_factors["Input Factors"] == "Max # Warehouses allowed"]["Data"].values[0])
max_warehouses

7

In [15]:
big_M_factor = float(input_factors.loc[input_factors["Input Factors"] == "big M Factor"]["Data"].values[0])
big_M_factor

500000.0

In [16]:
Transportation_Cost_per_unit_Per_Km	 = float(input_factors.loc[input_factors["Input Factors"] == "Transportation Cost per unit Per Km"]["Data"].values[0])
Transportation_Cost_per_unit_Per_Km

0.02

## Model Formulation

Our goal is to optimiz the fixed cost, variable cost and transportation cost to do this, will create the binary variable to open each warehouse(1 = Open warehouse) and use the open warehouse values as linear coefficients in the objective

In [17]:
# Creating a model
model = gp.Model("Warehouse_Optimization")

Restricted license - for non-production use only - expires 2025-11-24


### define the decission variable

Initially define binary type decission variable named 'open' in warehouse_data set

In [18]:
open_warehouse_ver = (warehouse_data.gppd.add_vars(model, vtype=GRB.BINARY, name="open"))
open_warehouse_ver

,fixed_cost,var_cost,cap,open
origin,,,,
chennai,50000,2.23,400000,<gurobi.Var *Awaiting Model Update*>
bhopal,56000,2.32,240000,<gurobi.Var *Awaiting Model Update*>
jaipur,65000,2.25,300000,<gurobi.Var *Awaiting Model Update*>
kolkata,65000,2.45,200000,<gurobi.Var *Awaiting Model Update*>
nasik,55000,2.21,300000,<gurobi.Var *Awaiting Model Update*>
pune,89000,2.56,300000,<gurobi.Var *Awaiting Model Update*>
sonipat,95000,2.78,240000,<gurobi.Var *Awaiting Model Update*>


Morover define one more binary type decission variable named 'flow' in Orig_Dest data set and in addition define the integer variable named 'unit_flow' in Orig_Dest data set

In [19]:
flow_vars = (
    Orig_Dest
    .gppd.add_vars(model, vtype=GRB.BINARY, name="flow")
    .gppd.add_vars(model, vtype=GRB.INTEGER, name="units_flow")
    .set_index(["origin", "destination"])
)
flow_vars

kms                                  flow  \
origin  destination                                                  
chennai chennai        100.0  <gurobi.Var *Awaiting Model Update*>   
        bhopal        1170.1  <gurobi.Var *Awaiting Model Update*>   
        bhubaneshwar   994.8  <gurobi.Var *Awaiting Model Update*>   
        jaipur        1605.6  <gurobi.Var *Awaiting Model Update*>   
        kochi          556.1  <gurobi.Var *Awaiting Model Update*>   
...                      ...                                   ...   
sonipat mumbai        1179.8  <gurobi.Var *Awaiting Model Update*>   
        nasik         1052.0  <gurobi.Var *Awaiting Model Update*>   
        new delhi       45.6  <gurobi.Var *Awaiting Model Update*>   
        pune          1207.8  <gurobi.Var *Awaiting Model Update*>   
        sonipat        100.0  <gurobi.Var *Awaiting Model Update*>   

                                                units_flow  
origin  destination                                         
chennai chennai       <gurobi.Var *Awaiting Model Update*>  
        bhopal        <gurobi.Var *Awaiting Model Update*>  
        bhubaneshwar  <gurobi.Var *Awaiting Model Update*>  
        jaipur        <gurobi.Var *Awaiting Model Update*>  
        kochi         <gurobi.Var *Awaiting Model Update*>  
...                                                    ...  
sonipat mumbai        <gurobi.Var *Awaiting Model Update*>  
        nasik         <gurobi.Var *Awaiting Model Update*>  
        new delhi     <gurobi.Var *Awaiting Model Update*>  
        pune          <gurobi.Var *Awaiting Model Update*>  
        sonipat       <gurobi.Var *Awaiting Model Update*>  

[90 rows x 3 columns]

Set the square root of decission variable

In [20]:
sum_open = model.addVar(vtype="I")
sqrt_sum_open = model.addVar()

### Add constraint


In [21]:
M_open1 = open_warehouse_ver["open"] * big_M_factor
M_open1

origin
chennai    500000.0 <gurobi.Var *Awaiting Model Update*>
bhopal     500000.0 <gurobi.Var *Awaiting Model Update*>
jaipur     500000.0 <gurobi.Var *Awaiting Model Update*>
kolkata    500000.0 <gurobi.Var *Awaiting Model Update*>
nasik      500000.0 <gurobi.Var *Awaiting Model Update*>
pune       500000.0 <gurobi.Var *Awaiting Model Update*>
sonipat    500000.0 <gurobi.Var *Awaiting Model Update*>
Name: open, dtype: object

In [60]:
# sum of the units flow and groupby wtih respect to the origin
sumif_units_flow = flow_vars['units_flow'].groupby('origin').sum()
sumif_units_flow

origin
bhopal     units_flow[13] + units_flow[14] + units_flow[1...
chennai    units_flow[0] + units_flow[1] + units_flow[2] ...
jaipur     units_flow[26] + units_flow[27] + units_flow[2...
kolkata    units_flow[39] + units_flow[40] + units_flow[4...
nasik      units_flow[52] + units_flow[53] + units_flow[5...
pune       units_flow[65] + units_flow[66] + units_flow[6...
sonipat    units_flow[78] + units_flow[79] + units_flow[8...
Name: units_flow, dtype: object

In [23]:
# unit_flow constraint
unit_flow_cons = gppd.add_constrs(
    model,
    M_open1,
    GRB.GREATER_EQUAL, sumif_units_flow,
    name = "unit_flow_cons"
    )
unit_flow_cons

origin
bhopal     <gurobi.Constr *Awaiting Model Update*>
chennai    <gurobi.Constr *Awaiting Model Update*>
jaipur     <gurobi.Constr *Awaiting Model Update*>
kolkata    <gurobi.Constr *Awaiting Model Update*>
nasik      <gurobi.Constr *Awaiting Model Update*>
pune       <gurobi.Constr *Awaiting Model Update*>
sonipat    <gurobi.Constr *Awaiting Model Update*>
Name: unit_flow_cons, dtype: object

In [61]:
# sum of flow and groupby with respect ot the origin
sumif_flow = flow_vars['flow'].groupby('origin').sum()
sumif_flow

origin
bhopal     flow[13] + flow[14] + flow[15] + flow[16] + fl...
chennai    flow[0] + flow[1] + flow[2] + flow[3] + flow[4...
jaipur     flow[26] + flow[27] + flow[28] + flow[29] + fl...
kolkata    flow[39] + flow[40] + flow[41] + flow[42] + fl...
nasik      flow[52] + flow[53] + flow[54] + flow[55] + fl...
pune       flow[65] + flow[66] + flow[67] + flow[68] + fl...
sonipat    flow[78] + flow[79] + flow[80] + flow[81] + fl...
Name: flow, dtype: object

In [25]:
# flow constraint
flow_cons_1 = gppd.add_constrs(
    model,
    M_open1,
    GRB.GREATER_EQUAL, sumif_flow,
    name = "flow_cons_1"
    )
flow_cons_1

origin
bhopal     <gurobi.Constr *Awaiting Model Update*>
chennai    <gurobi.Constr *Awaiting Model Update*>
jaipur     <gurobi.Constr *Awaiting Model Update*>
kolkata    <gurobi.Constr *Awaiting Model Update*>
nasik      <gurobi.Constr *Awaiting Model Update*>
pune       <gurobi.Constr *Awaiting Model Update*>
sonipat    <gurobi.Constr *Awaiting Model Update*>
Name: flow_cons_1, dtype: object

In [62]:
M_open2 = flow_vars["flow"] * big_M_factor
M_open2

origin   destination 
chennai  chennai          500000.0 flow[0]
         bhopal           500000.0 flow[1]
         bhubaneshwar     500000.0 flow[2]
         jaipur           500000.0 flow[3]
         kochi            500000.0 flow[4]
                               ...        
sonipat  mumbai          500000.0 flow[85]
         nasik           500000.0 flow[86]
         new delhi       500000.0 flow[87]
         pune            500000.0 flow[88]
         sonipat         500000.0 flow[89]
Name: flow, Length: 90, dtype: object

In [63]:
# flow constraint
flow_cons_2 = gppd.add_constrs(
    model,
    M_open2,
    GRB.GREATER_EQUAL, flow_vars['units_flow'],
    name = "flow_cons_2"
    )
flow_cons_2

origin   destination 
chennai  chennai         <gurobi.Constr *Awaiting Model Update*>
         bhopal          <gurobi.Constr *Awaiting Model Update*>
         bhubaneshwar    <gurobi.Constr *Awaiting Model Update*>
         jaipur          <gurobi.Constr *Awaiting Model Update*>
         kochi           <gurobi.Constr *Awaiting Model Update*>
                                          ...                   
sonipat  mumbai          <gurobi.Constr *Awaiting Model Update*>
         nasik           <gurobi.Constr *Awaiting Model Update*>
         new delhi       <gurobi.Constr *Awaiting Model Update*>
         pune            <gurobi.Constr *Awaiting Model Update*>
         sonipat         <gurobi.Constr *Awaiting Model Update*>
Name: flow_cons_2, Length: 90, dtype: object

In [28]:
# Demand constraint
Demand_cons = gppd.add_constrs(
    model,
    flow_vars['units_flow'].groupby('destination').sum(),
    GRB.GREATER_EQUAL, demand_data['ask_dem'],
    name = "Demand_cons"
    )
Demand_cons

destination
bhopal          <gurobi.Constr *Awaiting Model Update*>
bhubaneshwar    <gurobi.Constr *Awaiting Model Update*>
chennai         <gurobi.Constr *Awaiting Model Update*>
jaipur          <gurobi.Constr *Awaiting Model Update*>
kochi           <gurobi.Constr *Awaiting Model Update*>
kolkata         <gurobi.Constr *Awaiting Model Update*>
lucknow         <gurobi.Constr *Awaiting Model Update*>
mumbai          <gurobi.Constr *Awaiting Model Update*>
nasik           <gurobi.Constr *Awaiting Model Update*>
new delhi       <gurobi.Constr *Awaiting Model Update*>
pune            <gurobi.Constr *Awaiting Model Update*>
sonipat         <gurobi.Constr *Awaiting Model Update*>
srinagar        <gurobi.Constr *Awaiting Model Update*>
Name: Demand_cons, dtype: object

In [29]:
# capacity constrained
Capa_cons = gppd.add_constrs(
    model,
    flow_vars['units_flow'].groupby('origin').sum(),
    GRB.LESS_EQUAL, warehouse_data['cap'],
    name = "Capa_cons"
    )
Capa_cons

origin
bhopal     <gurobi.Constr *Awaiting Model Update*>
chennai    <gurobi.Constr *Awaiting Model Update*>
jaipur     <gurobi.Constr *Awaiting Model Update*>
kolkata    <gurobi.Constr *Awaiting Model Update*>
nasik      <gurobi.Constr *Awaiting Model Update*>
pune       <gurobi.Constr *Awaiting Model Update*>
sonipat    <gurobi.Constr *Awaiting Model Update*>
Name: Capa_cons, dtype: object

In [30]:
# add the constrained to set the value (√(∑(open)))
x = model.addConstr(sum_open == open_warehouse_ver["open"].sum())
y = model.addConstr(sum_open == sqrt_sum_open**2)

Define the objective of model -- to minimize the fixed cost, variable cost and transportaion cost

In [31]:
total_fixed_cost = gp.quicksum(warehouse_data["fixed_cost"] * open_warehouse_ver["open"])
total_fixed_cost

<gurobi.LinExpr: 50000.0 <gurobi.Var *Awaiting Model Update*> + 56000.0 <gurobi.Var *Awaiting Model Update*> + 65000.0 <gurobi.Var *Awaiting Model Update*> + 65000.0 <gurobi.Var *Awaiting Model Update*> + 55000.0 <gurobi.Var *Awaiting Model Update*> + 89000.0 <gurobi.Var *Awaiting Model Update*> + 95000.0 <gurobi.Var *Awaiting Model Update*>>

In [32]:
# sum of unit flow and groupby with respect ot origin
sumif_units_flow = flow_vars['units_flow'].groupby('origin').sum()
sumif_units_flow

origin
bhopal     <gurobi.Var *Awaiting Model Update*> + <gurobi...
chennai    <gurobi.Var *Awaiting Model Update*> + <gurobi...
jaipur     <gurobi.Var *Awaiting Model Update*> + <gurobi...
kolkata    <gurobi.Var *Awaiting Model Update*> + <gurobi...
nasik      <gurobi.Var *Awaiting Model Update*> + <gurobi...
pune       <gurobi.Var *Awaiting Model Update*> + <gurobi...
sonipat    <gurobi.Var *Awaiting Model Update*> + <gurobi...
Name: units_flow, dtype: object

In [33]:
total_var_cost = gp.quicksum(warehouse_data['var_cost'] * sumif_units_flow)
total_var_cost

<gurobi.LinExpr: 2.32 <gurobi.Var *Awaiting Model Update*> + 2.32 <gurobi.Var *Awaiting Model Update*> + 2.32 <gurobi.Var *Awaiting Model Update*> + 2.32 <gurobi.Var *Awaiting Model Update*> + 2.32 <gurobi.Var *Awaiting Model Update*> + 2.32 <gurobi.Var *Awaiting Model Update*> + 2.32 <gurobi.Var *Awaiting Model Update*> + 2.32 <gurobi.Var *Awaiting Model Update*> + 2.32 <gurobi.Var *Awaiting Model Update*> + 2.32 <gurobi.Var *Awaiting Model Update*> + 2.32 <gurobi.Var *Awaiting Model Update*> + 2.32 <gurobi.Var *Awaiting Model Update*> + 2.32 <gurobi.Var *Awaiting Model Update*> + 2.23 <gurobi.Var *Awaiting Model Update*> + 2.23 <gurobi.Var *Awaiting Model Update*> + 2.23 <gurobi.Var *Awaiting Model Update*> + 2.23 <gurobi.Var *Awaiting Model Update*> + 2.23 <gurobi.Var *Awaiting Model Update*> + 2.23 <gurobi.Var *Awaiting Model Update*> + 2.23 <gurobi.Var *Awaiting Model Update*> + 2.23 <gurobi.Var *Awaiting Model Update*> + 2.23 <gurobi.Var *Awaiting Model Update*> + 2.23 <gurobi.Va

In [34]:
total_Trans_cost = gp.quicksum(flow_vars['kms'] * flow_vars['units_flow'] * Transportation_Cost_per_unit_Per_Km)
total_Trans_cost

<gurobi.LinExpr: 2.0 <gurobi.Var *Awaiting Model Update*> + 23.401999999999997 <gurobi.Var *Awaiting Model Update*> + 19.896 <gurobi.Var *Awaiting Model Update*> + 32.112 <gurobi.Var *Awaiting Model Update*> + 11.122 <gurobi.Var *Awaiting Model Update*> + 27.162 <gurobi.Var *Awaiting Model Update*> + 30.651999999999997 <gurobi.Var *Awaiting Model Update*> + 20.67 <gurobi.Var *Awaiting Model Update*> + 20.692 <gurobi.Var *Awaiting Model Update*> + 35.12 <gurobi.Var *Awaiting Model Update*> + 18.296 <gurobi.Var *Awaiting Model Update*> + 36.012 <gurobi.Var *Awaiting Model Update*> + 48.018 <gurobi.Var *Awaiting Model Update*> + 23.401999999999997 <gurobi.Var *Awaiting Model Update*> + 2.0 <gurobi.Var *Awaiting Model Update*> + 18.614 <gurobi.Var *Awaiting Model Update*> + 8.746 <gurobi.Var *Awaiting Model Update*> + 29.634 <gurobi.Var *Awaiting Model Update*> + 22.484 <gurobi.Var *Awaiting Model Update*> + 10.712000000000002 <gurobi.Var *Awaiting Model Update*> + 13.218 <gurobi.Var *Awai

In [35]:
# sum of flow in "Orig_dst" data set
sum_of_flow =  gp.quicksum(flow_vars["flow"])

Inventry cost

In [36]:
# sum of ask_dem from demand data
ask_dem_yr = gp.quicksum(demand_data["ask_dem"])
ask_dem_yr

<gurobi.LinExpr: 443000.0>

In [37]:
# weekly demand
avg_dem_week = ask_dem_yr/52
avg_dem_week

<gurobi.LinExpr: 8519.23076923077>

In [38]:
Avg_Review_Period = int(input_factors.loc[input_factors["Input Factors"] == "Avg Review Period"]["Data"].values[0])
Avg_Review_Period

12

In [39]:
cycle_stock_units = avg_dem_week * Avg_Review_Period
cycle_stock_units

<gurobi.LinExpr: 102230.76923076923>

In [40]:
Avg_Lead_Time = int(input_factors.loc[input_factors["Input Factors"] == "Avg Lead Time"]["Data"].values[0])
Avg_Lead_Time

8

In [41]:
Avg_Safety_Stock = int(input_factors.loc[input_factors["Input Factors"] == "Avg Safety Stock"]["Data"].values[0])
Avg_Safety_Stock

4

In [42]:
ss_unit = Avg_Safety_Stock * avg_dem_week
ss_unit

<gurobi.LinExpr: 34076.92307692308>

In [43]:
Avg_Unit_Price = int(input_factors.loc[input_factors["Input Factors"] == "Avg Unit Price"]["Data"].values[0])
Avg_Unit_Price

150

In [44]:
avg_cycle_inv = (cycle_stock_units / 2) * Avg_Unit_Price
print(f'avg cycle inv Cost: ${avg_cycle_inv}')
avg_cycle_inv

avg cycle inv Cost: $7667307.692307693


<gurobi.LinExpr: 7667307.692307693>

In [45]:
avg_ss = (ss_unit / 2) * Avg_Unit_Price
print(f'avg ss Cost: ${avg_ss}')
avg_ss

avg ss Cost: $2555769.230769231


<gurobi.LinExpr: 2555769.230769231>

In [46]:
avg_inventory_holding_cost = input_factors.loc[input_factors['Input Factors'] == 'Avg Inventory Holding Cost', 'Data'].values[0]
avg_inventory_holding_cost

0.15

In [47]:
inv_hol_cst = (avg_cycle_inv + avg_ss) * avg_inventory_holding_cost
print(f'Inv hol Cost: ${avg_ss}')
inv_hol_cst

Inv hol Cost: $2555769.230769231


<gurobi.LinExpr: 1533461.5384615385>

In [58]:
# total inventory hole cost
inv_hol_cst_total = sqrt_sum_open * inv_hol_cst
inv_hol_cst_total

<gurobi.LinExpr: 1533461.5384615385 C188>

In [49]:
obj1 = model.setObjectiveN(total_fixed_cost + total_var_cost + total_Trans_cost + inv_hol_cst_total, index=0, priority=0, name="obj1")

In [50]:
obj2 = model.setObjectiveN(sum_of_flow, index=1, priority=1, name="obj2")

## Step 6: Solve the model

``` python
Model.optimize() # optimize the model

Model.write(filename) # write model to a file
```

In [51]:
model.optimize() # optimize the model

model.write("linear_model.lp") # write model to a file

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 125 rows, 189 columns and 562 nonzeros
Model fingerprint: 0x4edf394c
Model has 1 quadratic constraint
Variable types: 1 continuous, 188 integer (97 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+05]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+04, 4e+05]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 

In [52]:
total_tran_cost_value = total_Trans_cost.getValue()
print(f'Total transportation  Cost: ${total_tran_cost_value}')

Total transportation  Cost: $2448412.0


In [53]:
total_var_cost_value = total_var_cost.getValue()
print(f'Total variable  Cost: ${total_var_cost_value}')

Total variable  Cost: $1004590.0


In [54]:
total_fixed_cost_value = total_fixed_cost.getValue()
print(f'Total Fixed Cost: ${total_fixed_cost_value}')

Total Fixed Cost: $235000.0


In [55]:
inv_hol_cst_value = inv_hol_cst_total.getValue()
print(f'Total inventory hole Cost: ${inv_hol_cst_value}')

Total inventory hole Cost: $3066923.076923077


In [56]:
Total_Coast = total_fixed_cost_value + total_var_cost_value + total_tran_cost_value + inv_hol_cst_value
Total_Coast
print(f'Total Cost: ${Total_Coast}')

Total Cost: $6754925.076923077


In [57]:
# Create a dictionary with your values
data = {'Total transportation': [total_tran_cost_value], 'Total variable': [total_var_cost_value], 'Total Fixed': [total_fixed_cost_value],"Total inventory": [inv_hol_cst_value],'Total': [Total_Coast]}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)
# Set a new index (for example, using a list of labels)
new_index = ['Cost $']

# Use set_index to change the index of the DataFrame
Total_cost = df.set_index(pd.Index(new_index))
Total_cost

,Total transportation,Total variable,Total Fixed,Total inventory,Total
Cost $,2448412.0,1004590.0,235000.0,3.066923e+06,6.754925e+06
